In [1]:
from tile_match_gym.board import Board
from copy import deepcopy
import numpy as np
from itertools import product
from multiprocessing import Pool

In [2]:
import numpy as np
from tile_match_gym.board import Board
from typing import List, Tuple
from copy import deepcopy


# create an array of alternating 2's and 3's
def create_alternating_array(height: int, width: int) -> np.ndarray:
    arr = np.ones((2, height, width), dtype=int)
    for i in range(height):
        for j in range(width):
            arr[0, i, j] = 2 - int((i % 2) == (j % 2))
    return arr 


def create_board_from_array(arr: np.ndarray, num_colours=4) -> Board:
    
    height, width, _ = arr.shape
    seed = 1
    board = Board(height, width, num_colours, ["cookie"], ["vertical_laser", "horizontal_laser", "bomb"], seed)
    # board.board[0] = deepcopy(arr)
    # board.board[1] = np.ones_like(arr)
    return board


def create_alternating_board(height: int, width: int) -> Board:
    arr = create_alternating_array(height, width)
    return create_board_from_array(arr)


def contains_threes(arr: np.ndarray) -> bool:
    rows, cols = arr.shape
    for i in range(rows):
        for j in range(cols):
            if j < cols - 2 and arr[0, i, j] == arr[0, i, j + 1] == arr[0, i, j + 2] != 0:
                return True
            if i < rows - 2 and arr[0, i, j] == arr[0, i + 1, j] == arr[0, i + 2, j] !=0:
                return True
    return False


def wipe_coords(board: Board, coords: List[Tuple[int, int]]) -> np.ndarray:    
    for coord in coords:
        board.board[:, coord[0], coord[1]] *= 0


def get_special_locations(board: Board) -> List[Tuple[int, int]]:
    locations = []
    for i in range(board.num_rows):
        for j in range(board.num_cols):
            if board.board[1, i, j] not in [0, 1]:
                locations.append((i, j))
    return locations


In [3]:
import numpy as np
from tile_match_gym.board import Board
from copy import deepcopy


# 3 in a row and 3 in a col
board = Board(10, 10, 5) 
board.board[0] = np.array([[4, 4, 5, 4, 2, 2, 6, 6, 3, 3],
                            [3, 6, 3, 6, 3, 4, 5, 4, 2, 2],
                            [6, 5, 6, 4, 6, 3, 4, 5, 2, 3],
                            [2, 4, 6, 2, 3, 4, 6, 3, 4, 3],
                            [2, 5, 2, 3, 4, 4, 2, 6, 5, 6],
                            [4, 5, 3, 4, 6, 3, 5, 2, 5, 6],
                            [4, 4, 3, 2, 4, 5, 4, 5, 3, 5],
                            [5, 6, 4, 2, 5, 4, 6, 4, 3, 2],
                            [4, 5, 5, 6, 3, 4, 6, 3, 5, 6],
                            [4, 4, 5, 4, 6, 5, 2, 2, 4, 6]])

old_board = deepcopy(board.board)
assert board.check_move_validity((1,1), (1, 2))
assert board.check_move_validity((1,1), (2, 1))
assert np.array_equal(board.board, old_board)

# 3 in a col
board = Board(4, 3, 4)
board.board[0] = np.array([[2, 4, 3],
                           [2, 1, 3],
                           [1, 3, 1],
                           [1, 4, 1]])

old_board = deepcopy(board.board)
assert board.check_move_validity((2, 1), (2, 2))
assert np.array_equal(board.board, old_board)
assert board.check_move_validity((1, 1), (1, 0))
assert np.array_equal(board.board, old_board)
assert board.check_move_validity((1, 1), (1, 2))
assert np.array_equal(board.board, old_board)

# 3 in a col with special.
board.board[1] = np.array([[1, 1, 1],
                           [2, 1, 3],
                           [1, 1, 1],
                           [1, 1, 2]])

old_board = deepcopy(board.board)
assert board.check_move_validity((2, 1), (2, 2))
assert np.array_equal(board.board, old_board)


# 4 in a row with special
board = Board(4, 5, 5)
board.board[0] = np.array([[1, 2, 2, 3, 4],
                           [1, 2, 1, 2, 4],
                           [4, 4, 1, 4, 2],
                           [1, 1, 4, 3, 2]]) 

board.board[1] = np.array([[1, 1, 1, 1, 1],
                           [1, 1, 1, 1, 1],
                           [1, 3, 1, 1, 1],
                           [1, 1, 2, 1, 1]]) 
old_board = deepcopy(board.board)
assert board.check_move_validity((2, 2), (3, 2))
assert np.array_equal(board.board, old_board)

# 4 in a column
board.board[0] = np.array([[1, 2, 2, 3, 4],
                           [1, 2, 1, 2, 4],
                           [4, 4, 2, 4, 2],
                           [1, 2, 3, 3, 2]]) 

board.board[1] = np.array([[1, 1, 1, 1, 1],
                           [1, 1, 1, 1, 1],
                           [1, 3, 1, 1, 1],
                           [1, 3, 2, 1, 1]]) 

old_board = deepcopy(board.board)
assert board.check_move_validity((2, 1), (2, 2))
assert np.array_equal(board.board, old_board)

# 5 in a col
board = Board(5, 5, 7, seed=2)  # cookie , and vertical lase, horizontal laser, bomb

board.board[0] = np.array([[1, 2, 3, 3, 4],
                           [1, 2, 3, 2, 4],
                           [4, 4, 2, 3, 2],
                           [1, 2, 3, 2, 2],
                           [1, 2, 3, 1, 2]]) 

board.board[1] = np.array([[1, 1, 1, 2, 1],
                           [1, 1, 1, 1, 1],
                           [1, 3, 1, 1, 1],
                           [1, 3, 2, 1, 1],
                           [1, 2, 3, 3, 2]])
old_board = deepcopy(board.board)
assert board.check_move_validity((2, 3), (2, 2))
assert np.array_equal(board.board, old_board)

# Cookie + special
board = Board(7, 5, 12, seed=3)
print(board.board)
board.board[0] = np.array([[8, 2, 9, 4, 5], 
                           [1, 9, 10, 6, 8], 
                           [4, 6, 11, 4, 9], 
                           [12, 11, 3, 4, 6], 
                           [8, 10, 8, 7, 8], 
                           [12, 1, 6, 6, 2], 
                           [4, 1, 12, 9, 12]])

board.board[1] = np.array([[1, 1, 1, 1, 1], 
                           [1, 1, 1, 1, 1], 
                           [1, 1, 1, 1, 1], 
                           [1, 1, 3, 1, 1], 
                           [1, 1, -1, 1, 1], 
                           [1, 1, 1, 1, 1], 
                           [1, 1, 1, 1, 1]])
board.board[0, 4, 2] = 0
old_board = deepcopy(board.board)
assert board.check_move_validity((3, 2), (4, 2))
assert np.array_equal(board.board, old_board)


# Cookie + normal
board.board[0, 4, 2] = 8
board.board[0, 0, 0] = 0
board.board[1] = np.array([[-1, 1, 1, 1, 1], 
                           [1, 1, 1, 1, 1], 
                           [1, 1, 1, 1, 1], 
                           [1, 1, 1, 1, 1], 
                           [1, 1, 1, 1, 1], 
                           [1, 1, 1, 1, 1], 
                           [1, 1, 1, 1, 1]])

old_board = deepcopy(board.board)
assert board.check_move_validity((0, 0), (0, 1))
assert board.check_move_validity((0, 0), (1, 0))
assert np.array_equal(board.board, old_board)


# Cookie + cookie
board.board[0, 0, 0] = 8
board.board[0, 6, 4] = 0
board.board[0, 6, 3] = 0

board.board[1] = np.array([[-1, 1, 1, 1, 1], 
                           [1, 1, 1, 1, 1], 
                           [1, 1, 1, 1, 1], 
                           [1, 1, 1, 1, 1], 
                           [1, 1, 1, 1, 1], 
                           [1, 1, 1, 1, 1], 
                           [1, 1, 1, -1, -1]])

old_board = deepcopy(board.board)
assert board.check_move_validity((6, 4), (6, 3))
assert np.array_equal(board.board, old_board)


# Not adjacent
assert not board.check_move_validity((0, 0), (0, 2))
assert not board.check_move_validity((2, 3), (3, 4))
assert not board.check_move_validity((0, 0), (6, 5))

# Same coordinate
assert not board.check_move_validity((0, 0), (0, 0))
assert not board.check_move_validity((3, 3), (3, 3))


[[[ 8  2  9  4  5]
  [ 1  9 10  6  8]
  [ 4  6 11  4  9]
  [12 11  3  4  6]
  [ 8 10  8  7  8]
  [12  1  6  6  2]
  [ 4  1 12  9 12]]

 [[ 1  1  1  1  1]
  [ 1  1  1  1  1]
  [ 1  1  1  1  1]
  [ 1  1  1  1  1]
  [ 1  1  1  1  1]
  [ 1  1  1  1  1]
  [ 1  1  1  1  1]]]


In [64]:
def process_combination(combination):
    count_s, count_s_, count_m = 0, 0, 0
    b = Board(4, 4, 3,[], [])
    b.board = np.array(combination).reshape(4, 4)
    no_poss_move = not b.possible_move()
    has_match = len(b.get_colour_lines()) > 0
    if has_match or no_poss_move:
        count_s = 1
    if not has_match and no_poss_move:
        count_s_= 1
    if has_match:
        count_m = 1

    return count_s, count_s_, count_m

In [65]:
num_invalid_s, num_valid_states_no_move, num_states_w_move_and_match = 0, 0, 0
with Pool() as pool:
    results = pool.map(process_combination, x)

    # Aggregate results from parallel processes

    for result in results:
        num_invalid_s += result[0]
        num_valid_states_no_move += result[1]
        num_states_w_move_and_match += result[2]

print(len(x) - num_invalid_s, num_valid_states_no_move, num_states_w_move_and_match)

8426718 57420 34562583


In [45]:
len(x)

531441

In [41]:
count_s

13303

In [33]:
count_s_

6377

In [34]:
count_m

2458

In [20]:
count_poss

6380

In [21]:
for i, combination in enumerate(x):
    b = Board(3, 3, 3, ["vertical_laser"], [])
    b.board = np.array(combination).reshape(3,3)
    no_poss_move = not b.possible_move()
    has_match = len(b.get_colour_lines()) > 0
    if has_match or no_poss_move:
        count_tot -= 1
    if has_match:
        count_m += 1
    if not no_poss_move:
        count_poss += 1

    # if i % 1000 == 1:
    #     print(i, count_m / i, count_poss / i, count_tot, b.board)

    
        


    

In [7]:
valid_arr

10077696

In [ ]:

board = Board(10, 10, 5)  # cookie , and vertical laser, horizontal laser, bomb
board.board = np.array([[4, 4, 5, 4, 2, 2, 6, 6, 3, 3],
                        [3, 6, 3, 6, 3, 4, 5, 4, 2, 2],
                        [6, 5, 6, 4, 6, 3, 4, 5, 2, 3],
                        [2, 4, 6, 2, 3, 4, 6, 3, 4, 3],
                        [2, 5, 2, 3, 4, 4, 2, 6, 5, 6],
                        [4, 5, 3, 4, 6, 3, 5, 2, 5, 6],
                        [4, 4, 3, 2, 4, 5, 4, 5, 3, 5],
                        [5, 6, 4, 2, 5, 4, 6, 4, 3, 2],
                        [4, 5, 5, 6, 3, 4, 6, 3, 5, 6],
                        [4, 4, 5, 4, 6, 5, 2, 2, 4, 6]])

old_board = deepcopy(board.board)
assert board.check_move_validity((1,1), (1, 2))
assert board.check_move_validity((1,1), (2, 1))
assert np.array_equal(board.board, old_board)